# Dev notebook

I use this notebook to develop my implementation of a simple and minimal neural network framework.

Inspiration for this network is drawn from the book [Neural Networks from Scratch (NNFS)](https://nnfs.io/) & [Pytorch's implementation](https://pytorch.org/)

Notes:
- The work on experimenting with backpropagation has been moved to another notebook to keep this one minimal
- I have chosen to use a similar naming convention to that used by pytorch (why reinvent the wheel), this has the benefit of ensuring that when we compare implementations the architecture of the networks is the same. 

In [141]:
# Import for dev and testing
import nnfs 
import numpy as np

import matplotlib.pyplot as plt

from abc import ABC, abstractmethod
from nnfs.datasets import spiral_data

nnfs.init()

## Defining Base Module

This is a module that contains all the base attributes and function needed by every class in the framework

In [142]:
class Module(ABC):
    """Base class for all classes in frame work to ensure the same attributes and common function names."""

    def __init__(self) -> None:
        # Attributes to hold input and outputs
        self.input = None
        self.output = None
    
    @abstractmethod
    def forward(self):
        pass

    @abstractmethod
    def backward(self):
        pass

##  Defining layers 

In this section We define and test the layers

Notes:
- since we intend to use ReLU as one of our activation functions we will use the He weight initialization method as described in https://arxiv.org/abs/1502.01852

### Linear Layer

In [143]:
class LinearLayer(Module):
    """Linear transformation layer of the type o = ixW + b,
    
    where I is the incoming vector, W is the layers weight matrix, b is bias vector and o is the dot product of the 
    i and W plus the bias
    
    Args:
        in_features (int): the size of the input features 
        out_features (int): the size of the output features
        
    Attributes:
        weights (np_array) numpy array of in_features x n_neurons
        biases  (np_array) numpy array of 1 x n_neurons
        inputs  (np_array) numpy array of latest batch of inputs
        inputs  (np_array) numpy array of latest batch of outputs
        d_w     (np_array) The current gradients with respect to the weights 
        d_x     (np_array) The current gradients with respect to the inputs
        d_b     (np_array) The current gradients with respect to the biases
    """

    def __init__(self, in_features, out_features) -> None:
        super().__init__()
        # initializing weights and biases 
        #self.weights = np.random.normal(0.0, np.sqrt(2/in_features), (in_features, out_features))
        # Using a simpler initialization  for testing 
        self.weights = 0.01 * np.random.randn(in_features, out_features)
        self.bias = np.zeros((1, out_features))

    def forward(self, inputs):
        # Saving inputs for backward step
        self.input = inputs
        self.output = np.dot(inputs, self.weights) + self.bias
        return self.output

    def backward(self, d_vals):
        """Backpropagation  of the linear function

        Args:
            d_vals (np_array) array of derivatives from the previous layer/function.
        """
        self.d_w = np.dot(self.input.T, d_vals)
        self.d_x = np.dot(d_vals, self.weights.T)
        self.d_b = np.sum(d_vals, axis=0, keepdims=True)


        

#### Testing Linear Layer

In [144]:
# The sample data is a list of coordinate, ie two points 
# The layer therefore will take 2 inputs 
# We have given it 3 out features (3 neurons) so we expect to see an output with the shape (n_samples*n_neurons, n_neurons)
# In out case that should be (300, 3) 
X, _ = spiral_data(samples=100, classes=3)
linear1 = LinearLayer(2, 3)
output = linear1.forward(X)
print(output[:10, :])

assert output.shape == (300,3)


[[ 0.0000000e+00  0.0000000e+00  0.0000000e+00]
 [-1.0475188e-04  1.1395361e-04 -4.7983500e-05]
 [-2.7414842e-04  3.1729150e-04 -8.6921798e-05]
 [-4.2188365e-04  5.2666257e-04 -5.5912682e-05]
 [-5.7707680e-04  7.1401405e-04 -8.9430439e-05]
 [-3.5430698e-04  3.5025488e-04 -2.3363481e-04]
 [-8.9267001e-04  1.0767876e-03 -1.9453237e-04]
 [-9.3350781e-04  1.0723802e-03 -3.1227397e-04]
 [-1.1243758e-03  1.3112801e-03 -3.3629674e-04]
 [-1.3386955e-03  1.6200906e-03 -2.8101794e-04]]


## Defining Activation Functions

### ReLu

$$y = \begin{cases}
   x &x> 0 \\
   0 & otherwise
\end{cases} $$

In [145]:
class ReLU(Module):
    """Applies Rectified linear Unit function to vector."""
    def __init__(self) -> None:
        # initializing attributes needed for backwards 
        super().__init__()
        self.d_relu = None
    
    def forward(self, x):
        # storing inputs needed for backwards 
        self.inputs = x
        self.outputs = np.maximum(x, 0)
        return self.outputs
    
    def backward(self, d_vals):
        self.d_relu = d_vals.copy()
        self.d_relu[self.inputs <= 0] = 0

#### Testing ReLU

In [146]:
i = [-2, 3, 4, 0, 0.1, -44]
test_relu = ReLU()

# Checking values are as expected 
assert np.all(np.array([0., 3, 4, 0, 0.1, 0.]) == test_relu.forward(i))

test_relu.outputs

array([0. , 3. , 4. , 0. , 0.1, 0. ])

### Softmax

$$\text{softmax}(x)_i = \frac{exp(x_i)}{\sum_{j}^{ }exp(x_j))}$$

The soft max represents the confidence score for each output class and adds up to 1.

In [147]:
class Softmax(Module):
    """Applies Softmax function to input matrix."""

    def __init__(self) -> None:
        super().__init__()
        self.confidence_scores = None

    def forward(self, x):
        # exponenets of each value
        exp_vals = np.exp(x - np.max(x, axis=1, keepdims=True))
        exp_sum = np.sum(exp_vals, axis=1, keepdims=True)
        # Normalization to get the proabilities 
        self.confidence_scores = exp_vals/exp_sum
        return self.confidence_scores

    def backward(self, d_vals):
        # Initialize array for gradients wrt to inputs
        self.d_soft = np.zeros_like(d_vals)
        
        _iter = enumerate(zip(self.confidence_scores, d_vals))
        for i, conf_score, d_val in _iter:
            # Flatten confidence scores
            cs = conf_score.reshape(-1, 1)
            # Find the Jacobian matrix of the output 
            j_matrix = np.diagflat(cs) - np.dot(cs, cs.T)
            # get the gradient 
            self.d_soft[i] = np.dot(j_matrix, d_val)
    
    def backward(self, y_pred, y_true):
        """Does a the combined backward pass for CCE & Softmax as a single, faster step."""
        n = len(y_pred)

        # Getting descrete vals from one hot encoding 
        y_true = np.argmax(y_true, axis=1)
        
        self.d_soft = y_pred.copy()
        self.d_soft[range(n), y_true] -= 1
        self.d_soft = self.d_soft/n
        return self.d_soft

#### Testing Softmax

In [148]:
softmax = Softmax()
softmax.forward([[1,2,44]])

array([[2.11513104e-19, 5.74952226e-19, 1.00000000e+00]])

## Defining Loss - Categorical Cross-Entropy

$$ L_i = -\sum_j y_{i,j}\log(\hat{y}_{i,j}) $$

With taking one hot encoding into account we can simplify this down to:

$$ L_i = -y_{i,k}\log(\hat{y}_{i,k}) $$

where K is the index of the correct class

In [149]:
class CategoricalCrossEntropyLoss:
    """Calculates the CCE loss for a given set of predictions.
    This method expect a softmax output and one-hot encoded label mask
    
    y_pred (np_array): matrix of confidence scores of the prediction
    y_true (np_array): matrix of one-hot encoded true lables of the classes
    """
    def forward(y_pred, y_true):
        # Clipping and applying one hot encoded labels as mask 
        # to zero out scores corresponding to incorrect classes
        # We clip to make sure that none of the reaming classes are 0 or 
        # exactly 1 
        corrected = np.sum(np.clip(y_pred, 1e-7, 1-1e-7)*y_true, axis=1)
        # Taking the -ve log of the remaining confidence scores 
        negative_log = -np.log(corrected)
        return np.mean(negative_log)
    
    def backward(y_pred, y_true):
        """Backpropagation  of the CCE Loss

        Args:
            y_pred (np_array) array of predictions.
            y_true (np_array) array of correct labels.
        """
        return (-y_true/y_pred)/len(y_pred)

#### Testing CCE Loss

In [150]:
y_pred = np.array([[0.7, 0.1, 0.2], [0.1,0.5,0.4],[0.02,0.9,0.08]])
y_true = np.array([[1,0,0], [0,1,0], [0,1,0]])

loss_function = CategoricalCrossEntropyLoss
loss_function.forward(y_pred, y_true)

0.38506088005216804

## Defining Utility functions

### One-hot encoding function 

In [151]:
def one_hot_encode_index(y, n):
    return np.eye(n)[y]

#### Testing one hot masker

In [152]:
n=3
y_test = np.array([0,1,2, 1, 2])

one_hot_encode_index(y_test, n)

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.]], dtype=float32)

### Define Accuracy 

In [153]:
def accuracy(y_pred, y_true):
    """Calculates the accuracy of a batch of predictions"""
    return np.mean(np.argmax(y_pred, axis=1) == np.argmax(y_true, axis=1))

## Integration Testing 

### Data initialization 

In [154]:
X, y = spiral_data(samples=100, classes=3)
y = one_hot_encode_index(y, 3)

In [155]:
# Initializing Network Components 
relu = ReLU()
softmax = Softmax()
cce_loss = CategoricalCrossEntropyLoss
linear1 = LinearLayer(2, 3)
linear2 = LinearLayer(3, 3)

In [156]:
# Forward Pass
linear1.forward(X) 
relu.forward(linear1.output)
linear2.forward(linear1.output)
y_pred = softmax.forward(linear2.output)
y_pred[:10, :]

array([[0.33333334, 0.33333334, 0.33333334],
       [0.33333287, 0.33333397, 0.3333332 ],
       [0.33333236, 0.33333457, 0.3333331 ],
       [0.33333167, 0.33333546, 0.3333329 ],
       [0.33333117, 0.33333606, 0.33333278],
       [0.33333188, 0.33333504, 0.33333308],
       [0.33332995, 0.3333377 , 0.3333324 ],
       [0.33332956, 0.33333808, 0.33333236],
       [0.3333296 , 0.33333796, 0.33333248],
       [0.33333033, 0.33333692, 0.33333275]], dtype=float32)

In [157]:
# Calculating Loss
cce_loss.forward(y_pred, y)

1.0986005

In [158]:
# Calculating accuracy 
accuracy(y_pred, y)

0.39

In [159]:
# Backward pass
softmax.backward(y_pred, y)
linear2.backward(softmax.d_soft)
relu.backward(linear2.d_x)
linear1.backward(relu.d_relu)

In [161]:
# Inspecting gradients
print(linear1.d_w)
print(linear1.d_b)
print(linear2.d_w)
print(linear2.d_b)

[[-1.1180256e-04 -2.8108768e-06 -3.9291775e-05]
 [-1.3544197e-04  2.7041102e-05  4.4713821e-04]]
[[ 4.2581928e-04  8.3322429e-06 -1.0214525e-03]]
[[-2.3060311e-05  1.5360911e-04 -1.3054881e-04]
 [ 3.8738308e-05  6.8804878e-04 -7.2678708e-04]
 [-1.4138591e-04  5.3566502e-04 -3.9427908e-04]]
[[ 5.8533624e-07 -9.6648000e-07  4.0419400e-07]]
